# SHAP

SHAP (SHapley Additive exPlanations) is a method used to explain the output of machine learning models.  
SHAP aims to explain how an input affects the output of the model, by showing the impact of each input feature on the output.  
When reading the SHAP values, you will see for each input feature how much it positively or negatively pushed the output to the answer we got, compared to the average base value of the dataset.

You can read more here: https://trustyai-explainability.github.io/trustyai-site/main/local-explainers.html

In [4]:
import pandas as pd
import random

import keras

Let's start by loading some artifacts.  
We will need:
- The ONNX model
- Our pre-and-post processing artifacts
    - scaler.pkl
    - label_encoder.pkl
- Some data
    - The training inputs, these will be used to get an average input for our dataset
    - The test data, these will be used to get a point we want to analyze

In [ ]:
onnx_session = rt.InferenceSession("../2-dev_datascience/models/jukebox/1/model.onnx", providers=rt.get_available_providers())
onnx_input_name = onnx_session.get_inputs()[0].name
onnx_output_name = onnx_session.get_outputs()[0].name


with open('../2-dev_datascience/models/jukebox/1/artifacts/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

with open('../2-dev_datascience/models/jukebox/1/artifacts/label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

with open('../2-dev_datascience/models/jukebox/1/artifacts/y_test.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

X_train = pd.read_parquet("../2-dev_datascience/models/jukebox/1/artifacts/X_train.parquet")
X_test = pd.read_parquet("../2-dev_datascience/models/jukebox/1/artifacts/X_test.parquet")

We arbitrarily choose the first data point (song) in our test data to be the data we want to test.  
In practice, you might choose the data point that you predict the worst on, or a data point that gave an unexpected answer.  
We also look at how our data point looks when normalized (after going through pre-processing). This is how it will look like going into the model

In [ ]:
point_to_explain = X_test.iloc[0:1]
point_to_explain

In [10]:
X_train

,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
644220,False,190560,0.841,0.783,6,-4.915,0,0.0432,0.1060,0.000002,0.0528,0.514,94.989
473163,False,146060,0.735,0.628,8,-5.572,0,0.2120,0.2580,0.000004,0.1000,0.192,103.925
585395,False,152078,0.760,0.747,0,-5.083,0,0.0282,0.0675,0.000003,0.3090,0.884,128.912
157601,True,170480,0.920,0.696,6,-3.356,0,0.0742,0.1830,0.000000,0.1490,0.545,106.966
839617,False,120600,0.858,0.677,8,-4.294,1,0.1180,0.1060,0.000000,0.2460,0.584,100.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568129,True,197920,0.708,0.737,1,-4.045,1,0.0436,0.0739,0.001620,0.0955,0.607,91.986
285391,False,180530,0.492,0.686,5,-6.970,0,0.1400,0.2800,0.000000,0.1370,0.392,109.802
815084,False,190560,0.841,0.783,6,-4.915,0,0.0432,0.1060,0.000002,0.0528,0.514,94.989
722116,False,220750,0.593,0.354,4,-8.270,1,0.0279,0.6980,0.000036,0.2930,0.241,120.010


In [15]:
output_names = label_encoder.classes_
output_names

TrustyAI SHAP explainer requires our model to have a pandas dataframe as an input, and numpy or pandas output, so we wrap our model in a pred() function that makes sure the input and output are converted properly. 

In [ ]:
def pred(x):
    pred = onnx_session.run([onnx_output_name], {onnx_input_name: x.to_numpy().astype(np.float32)})[0]
    return pd.DataFrame(pred, columns=output_names)

In [ ]:
from trustyai.model import Model
trustyai_model = Model(pred, dataframe_input=True, output_names=output_names)

Let's try to use our TrustyAI Model to predict the output of our data point we want to explain with SHAP.

In [ ]:
prediction = trustyai_model(normalize_dataframe(point_to_explain))
prediction

And with everything set up, we can create a SHAP explainer and let it analyze our data point!  
You can also note that we add 100 data points from our training dataset to the SHAPExplainer, this is used to calculate the average base values of our dataset. With this, we can see how much our interesting datapoint contributes to the prediction compared to what a "standard" value would.

In [ ]:
from trustyai.explainers import SHAPExplainer
explainer = SHAPExplainer(background=normalize_dataframe(X_train[:100]))

In [ ]:
explanations = explainer.explain(inputs=normalize_dataframe(point_to_explain),
                                 outputs=prediction,
                                 model=trustyai_model)

With our SHAP Explainer ready we can start looking at the results.

Let's choose a specific output country which we want to know how it got affected by the input values.  
CH is the country that we are supposed to get as the popular country for this input, so it's especially interesting to see the input's effect on that output.  
That being said, feel free to try with a few other countries and see what happens.  

In [ ]:
COUNTRY_OF_INTEREST = "CH"

First, we will get a table of values.  
Here we can see the **Mean Background Value** - this is the average base value we were talking about before.  
We can also see our **Value**, which is the normalized data point that we sent into the explainer. Red values are lower than the average value and green values are higher.  
Finally, we have the **SHAP Value**. These indicate how much that input feature had an effect on the output. Red indicates a negative contribution to the prediction while green a positive contribution. The larger the value, the larger the contribution.

In [ ]:
explanations.as_html()[COUNTRY_OF_INTEREST]

We can also visualize it as a candlestick plot, seeing how the different input features build up to the output value.

In [ ]:
from trustyai.visualizations.shap import SHAPViz
SHAPViz()._matplotlib_plot(explanations=explanations, output_name=COUNTRY_OF_INTEREST)